In [1]:
from functools import wraps, partial
import inspect

In [2]:
def SingletonX(cls):
    # 如果装饰在基类上, 则继承类的实例的类变量永远是基类的类变量, 而不能拥有自己的类变量
    assert inspect.isclass(cls), 'The Singleton decorator can only be applied to class'

    def run_only_once(func):
        """
         使__init__只运行一次
        """

        @wraps(func)
        def _wrapper_init(self, *args, **kwargs):
            if not hasattr(self, "___init__called"):
                self.___init__called = True
                
                return func(self, *args, **kwargs)

        return _wrapper_init

    def newfunc(clsobj, *args, **kwargs):
        """
        使用元编程实现单例
        """
#         print(clsobj.a)
        if not hasattr(clsobj, "__instance"):
            setattr(clsobj, "__instance", object.__new__(clsobj))
        return getattr(clsobj, "__instance")

    # 拦截__new__方法
    setattr(cls, "__new__", newfunc)
    # 由于在创建cls时，python会调用__new__来创建实例，然后再调用实例的__init__
    # 这样通过在__new__使用object__new__能确保每次实例化cls时均返回同一个实例。
    # 但是python还是会调用__new__的__init__，这样会导致__init__方法每实例一次就会调用一次
    # 这种行为不是我们需要的，因此我们通过run_on_once装饰器做一定的处理，使__init__只调用一次
    setattr(cls, "__init__", run_only_once(cls.__init__))
    return cls

In [3]:
# @SingletonX   
class P:
    a = 12

    def __init__(self, name=None, age=None):
        print("__init__!")
        self.name = name
        self.age = age
#         self.t = Person.t

def create_p():
    person1 = P('lucky', 24)
    person2 = P('lau', 26)
    person3 = P()
    print(id(person1))
    print(id(person2))
    print(id(person3))

create_p()

__init__!
__init__!
__init__!
140179078074720
140179078074776
140179078074832


In [4]:
@SingletonX
class Q(P):
    a = 24

In [7]:
@SingletonX
class R(Q):
    a = 36

In [8]:
@SingletonX
class X(R):
    a = 48
    
p = X()
print(p.a)

48
